### Read TPCH Data

In [1]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!
import pyarrow.parquet as pq

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql


In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///:default:

In [4]:
tables = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
SF=10

In [5]:
TGTDIR = f"/Users/adarshrp/Projects/tpch-data/sf{SF}"

for t in tables:
    sqlstr = f"""
    CREATE OR REPLACE VIEW {t} AS SELECT * FROM read_parquet("{TGTDIR}/{t}.parquet");
    """
    print(sqlstr)
    duckdb.sql(sqlstr)



    CREATE OR REPLACE VIEW customer AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/customer.parquet");
    

    CREATE OR REPLACE VIEW lineitem AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/lineitem.parquet");
    

    CREATE OR REPLACE VIEW nation AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/nation.parquet");
    

    CREATE OR REPLACE VIEW orders AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/orders.parquet");
    

    CREATE OR REPLACE VIEW part AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/part.parquet");
    

    CREATE OR REPLACE VIEW partsupp AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/partsupp.parquet");
    

    CREATE OR REPLACE VIEW region AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/tpch-data/sf10/region.parquet");
    

    CREATE OR REPLACE VIEW supplier AS SELECT * FROM read_parquet("/Users/adarshrp/Proje

In [6]:
%sql SELECT count(*) from nation;

,count_star()
0,25


In [7]:
sqlstr = """
SET threads TO 10;

SELECT L_RETURNFLAG, L_LINESTATUS, SUM(L_QUANTITY) AS SUM_QTY,
SUM(L_EXTENDEDPRICE) AS SUM_BASE_PRICE, SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)) AS SUM_DISC_PRICE,
SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)*(L_TAX + 1.0)) AS SUM_CHARGE, AVG(L_QUANTITY) AS AVG_QTY,
AVG(L_EXTENDEDPRICE) AS AVG_PRICE, AVG(L_DISCOUNT) AS AVG_DISC, COUNT(L_QUANTITY) AS COUNT_ORDER
FROM LINEITEM
--WHERE L_SHIPDATE <= '1998-12-01'
GROUP BY L_RETURNFLAG, L_LINESTATUS;
"""

%time duckdb.sql(sqlstr)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.06 s, sys: 165 ms, total: 4.23 s
Wall time: 4.34 s


┌──────────────┬──────────────┬─────────────┬───┬────────────────────┬──────────────────────┬─────────────┐
│ l_returnflag │ l_linestatus │   SUM_QTY   │ … │     AVG_PRICE      │       AVG_DISC       │ COUNT_ORDER │
│   varchar    │   varchar    │   double    │   │       double       │        double        │    int64    │
├──────────────┼──────────────┼─────────────┼───┼────────────────────┼──────────────────────┼─────────────┤
│ N            │ O            │ 764635193.0 │ … │  38233.85393407358 │ 0.050001036771283126 │    29987794 │
│ R            │ F            │ 377732830.0 │ … │ 38251.219273561284 │  0.04999679231663862 │    14808183 │
│ A            │ F            │ 377518399.0 │ … │   38237.1510089518 │ 0.050006574542790186 │    14804077 │
│ N            │ F            │   9851614.0 │ … │   38257.8106600817 │ 0.049973367737715525 │      385998 │
├──────────────┴──────────────┴─────────────┴───┴────────────────────┴──────────────────────┴─────────────┤
│ 4 rows                    

In [8]:
sqlstr = """
SELECT SUM(L_EXTENDEDPRICE) 
FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

CPU times: user 479 µs, sys: 154 µs, total: 633 µs
Wall time: 638 µs


┌──────────────────────┐
│ sum(L_EXTENDEDPRICE) │
│        double        │
├──────────────────────┤
│    2293813156772.914 │
└──────────────────────┘

In [9]:
sqlstr = """
SELECT count(*)
FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

CPU times: user 271 µs, sys: 67 µs, total: 338 µs
Wall time: 344 µs


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     59986052 │
└──────────────┘

In [10]:
sqlstr = """
SET threads TO 20;

SELECT MAX(L_RETURNFLAG), MAX(L_LINESTATUS), SUM(L_QUANTITY) AS SUM_QTY,
SUM(L_EXTENDEDPRICE) AS SUM_BASE_PRICE, SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)) AS SUM_DISC_PRICE,
SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)*(L_TAX + 1.0)) AS SUM_CHARGE, AVG(L_QUANTITY) AS AVG_QTY,
AVG(L_EXTENDEDPRICE) AS AVG_PRICE, AVG(L_DISCOUNT) AS AVG_DISC, COUNT(L_QUANTITY) AS COUNT_ORDER
FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 7.15 s, sys: 165 ms, total: 7.32 s
Wall time: 3.67 s


┌───────────────────┬───────────────────┬──────────────┬───┬───────────────────┬─────────────────────┬─────────────┐
│ max(L_RETURNFLAG) │ max(L_LINESTATUS) │   SUM_QTY    │ … │     AVG_PRICE     │      AVG_DISC       │ COUNT_ORDER │
│      varchar      │      varchar      │    double    │   │      double       │       double        │    int64    │
├───────────────────┼───────────────────┼──────────────┼───┼───────────────────┼─────────────────────┼─────────────┤
│ R                 │ O                 │ 1529738036.0 │ … │ 38239.10859766057 │ 0.05000117760660247 │    59986052 │
├───────────────────┴───────────────────┴──────────────┴───┴───────────────────┴─────────────────────┴─────────────┤
│ 1 rows                                                                                      10 columns (6 shown) │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [11]:
sqlstr = """
SET threads TO 20;

SELECT MAX(L_RETURNFLAG), MAX(L_LINESTATUS), SUM(L_QUANTITY) AS SUM_QTY, MAX(L_TAX), MAX(L_SHIPDATE),
SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)*(L_TAX + 1.0)) AS SUM_CHARGE
FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.49 s, sys: 194 ms, total: 4.68 s
Wall time: 4.73 s


┌───────────────────┬───────────────────┬──────────────┬────────────┬─────────────────┬────────────────────┐
│ max(L_RETURNFLAG) │ max(L_LINESTATUS) │   SUM_QTY    │ max(L_TAX) │ max(L_SHIPDATE) │     SUM_CHARGE     │
│      varchar      │      varchar      │    double    │   double   │      date       │       double       │
├───────────────────┼───────────────────┼──────────────┼────────────┼─────────────────┼────────────────────┤
│ R                 │ O                 │ 1529738036.0 │       0.08 │ 1998-12-01      │ 2266298704206.5015 │
└───────────────────┴───────────────────┴──────────────┴────────────┴─────────────────┴────────────────────┘

In [12]:
sqlstr = """
SET threads TO 20;

SELECT MAX(L_RETURNFLAG), MAX(L_LINESTATUS), SUM(L_QUANTITY) AS SUM_QTY, MAX(L_TAX),
SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)*(L_TAX + 1.0)) AS SUM_CHARGE
FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 7.13 s, sys: 170 ms, total: 7.3 s
Wall time: 3.64 s


┌───────────────────┬───────────────────┬──────────────┬────────────┬────────────────────┐
│ max(L_RETURNFLAG) │ max(L_LINESTATUS) │   SUM_QTY    │ max(L_TAX) │     SUM_CHARGE     │
│      varchar      │      varchar      │    double    │   double   │       double       │
├───────────────────┼───────────────────┼──────────────┼────────────┼────────────────────┤
│ R                 │ O                 │ 1529738036.0 │       0.08 │ 2266298704206.5015 │
└───────────────────┴───────────────────┴──────────────┴────────────┴────────────────────┘

In [13]:
sqlstr = """
SELECT COUNT(L_PARTKEY) FROM LINEITEM
"""

%time duckdb.sql(sqlstr)

CPU times: user 299 µs, sys: 88 µs, total: 387 µs
Wall time: 393 µs


┌──────────────────┐
│ count(L_PARTKEY) │
│      int64       │
├──────────────────┤
│         59986052 │
└──────────────────┘

In [14]:
%sql SELECT * FROM parquet_metadata('/Users/adarshrp/Projects/tpch-data/sf1/lineitem.parquet');



,file_name,row_group_id,row_group_num_rows,row_group_num_columns,row_group_bytes,column_id,file_offset,num_values,path_in_schema,type,...,stats_distinct_count,stats_min_value,stats_max_value,compression,encodings,index_page_offset,dictionary_page_offset,data_page_offset,total_compressed_size,total_uncompressed_size
0,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,0,10817519,6001215,l_orderkey,INT32,...,NaN,1,6000000,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE, PLAIN",NaN,4,1049144,10817515,23180840
1,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,1,25133853,6001215,l_partkey,INT32,...,NaN,1,200000,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,10817623,11617685,14316230,14315507
2,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,2,35689195,6001215,l_suppkey,INT32,...,NaN,1,10000,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,25133951,25173977,10555244,10554722
3,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,3,37124303,6001215,l_linenumber,INT32,...,NaN,1,7,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,35689293,35689337,1435010,2262593
4,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,4,41638147,6001215,l_quantity,DOUBLE,...,NaN,1.0,50.0,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,37124404,37124632,4513743,4513701
5,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,5,76451924,6001215,l_extendedprice,DOUBLE,...,NaN,901.0,104949.5,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE, PLAIN",NaN,41638262,42398447,34813662,48244713
6,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,6,79465121,6001215,l_discount,DOUBLE,...,NaN,-0.0,0.1,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,76452052,76452140,3013069,3012932
7,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,7,82478290,6001215,l_tax,DOUBLE,...,NaN,-0.0,0.08,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,79465236,79465309,3013054,3012915
8,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,8,83941794,6001215,l_returnflag,BYTE_ARRAY,...,NaN,A,R,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,82478400,82478431,1463394,1463263
9,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,6001215,16,192847547,9,84825878,6001215,l_linestatus,BYTE_ARRAY,...,NaN,F,O,SNAPPY,"PLAIN_DICTIONARY, PLAIN, RLE",NaN,83941877,83941903,884001,939398


In [17]:
%sql SELECT * FROM parquet_metadata('/Users/adarshrp/Projects/tpch-data/sf1/lineitem.parquet2') where path_in_schema = 'l_orderkey'


,file_name,row_group_id,row_group_num_rows,row_group_num_columns,row_group_bytes,column_id,file_offset,num_values,path_in_schema,type,...,stats_distinct_count,stats_min_value,stats_max_value,compression,encodings,index_page_offset,dictionary_page_offset,data_page_offset,total_compressed_size,total_uncompressed_size
0,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,0,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,1,122308,SNAPPY,PLAIN,NaN,NaN,4,185315,491551
1,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,1,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,122308,245474,SNAPPY,PLAIN,NaN,NaN,5611481,185963,491551
2,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,2,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,245475,368898,SNAPPY,PLAIN,NaN,NaN,11224506,186297,491551
3,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,3,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,368899,491329,SNAPPY,PLAIN,NaN,NaN,16837553,185180,491551
4,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,4,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,491329,613731,SNAPPY,PLAIN,NaN,NaN,22458166,185415,491551
5,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,5,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,613731,736740,SNAPPY,PLAIN,NaN,NaN,28100065,185636,491551
6,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,6,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,736740,859847,SNAPPY,PLAIN,NaN,NaN,33708251,185805,491551
7,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,7,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,859847,982721,SNAPPY,PLAIN,NaN,NaN,39352981,185837,491551
8,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,8,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,982722,1105953,SNAPPY,PLAIN,NaN,NaN,44969325,185710,491551
9,/Users/adarshrp/Projects/tpch-data/sf1/lineite...,9,122880,16,0,0,0,122880,l_orderkey,INT32,...,NaN,1105953,1228870,SNAPPY,PLAIN,NaN,NaN,50576595,185798,491551
